In [3]:
# install PySyft if not available (e.g on cloud)
# pip install syft

# import required libraries
import torch
import random
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Subset
from torchvision import datasets, transforms
from syft.frameworks.torch.differential_privacy import pate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# import data set
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

train_data = mnist_trainset.train_data
train_targets = mnist_trainset.train_labels

test_data = mnist_trainset.test_data
test_targets = mnist_trainset.test_labels

C:\Users\eazyd\Anaconda3\envs\private-ai\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
C:\Users\eazyd\Anaconda3\envs\private-ai\lib\site-packages\torchvision\datasets\mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
C:\Users\eazyd\Anaconda3\envs\private-ai\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\eazyd\Anaconda3\envs\private-ai\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [5]:
# define function to split training data into sub-datasets for a number of teachers
# returns nt sub-datasets to train the each teacher model
# TODO: split into train and test 
def split_train_data(train_data, num_teachers):
    teacher_loaders = []
    p, q = divmod(len(train_data), num_teachers)
    #split_indices = list((train_data[i * p + min(i, q):(i + 1) * p + min(i + 1, q)] for i in range(num_teachers)))
    split_indices = list(range((i * p + min(i, q)) , ((i + 1) * p + min(i+1, q))) for i in range(num_teachers))
    for j in range(len(split_indices)):
        subset_j = Subset(train_data, split_indices[j])
        loader_j = torch.utils.data.DataLoader(subset_j, batch_size=64, shuffle=True)
        teacher_loaders.append(loader_j)
    return teacher_loaders

In [13]:
# define function to take a chunk of the test data as private dataset
# returns reduced test data and private dataset in ratio: 0 < ratio < 1
# to split reduced_test_data:private_data in 80:20, use ratio 0.8
def split_test_data(test_data, ratio):    
    divide = int(len(test_data) * ratio)
    
    reduced_indices = range(0, divide)
    private_indices = range(divide, len(test_data))

    reduced_subset = Subset(test_data, reduced_indices)
    private_subset = Subset(test_data, private_indices)
    
    test_loader = torch.utils.data.DataLoader(reduced_subset, batch_size=64, shuffle=True)
    private_loader = torch.utils.data.DataLoader(private_subset, batch_size=64, shuffle=True)
    
    return test_loader, private_loader

In [7]:
# define class to build linear classifier models for each teacher
class Classifier(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5):
        ''' Builds a feedforward network with arbitrary hidden layers.
        
            Arguments
            ---------
            input_size: integer, size of the input layer
            output_size: integer, size of the output layer
            hidden_layers: list of integers, the sizes of the hidden layers
        
        '''
        super().__init__()
        # Input to a hidden layer
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # Add a variable number of more hidden layers
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        self.output = nn.Linear(hidden_layers[-1], output_size)
        self.dropout = nn.Dropout(p=drop_p)
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        for each in self.hidden_layers:
            x = F.relu(each(x))
            x = self.dropout(x)
        x = self.output(x)
        
        return F.log_softmax(x, dim=1)

In [8]:
# define function to train model given train and test datasets
def train(model, trainloader, testloader, criterion, optimizer, epochs=5, print_every=50):
    steps = 0
    running_loss = 0
    model.to(device)
    for e in range(epochs):
        model.train() # Model in training mode, grad & dropout is on
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            steps += 1
            images.resize_(images.size()[0], 784)
            optimizer.zero_grad()
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if steps % print_every == 0:
                model.eval() # Model in training mode, grad & dropout is off
                with torch.no_grad():
                    test_loss, accuracy = validation(model, testloader, criterion)
                
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
                
                running_loss = 0
                model.train() # Model in training mode, grad & dropout is on

In [9]:
# define function to validate model using the reduced test data set
def validation(model, testloader, criterion):
    accuracy = 0
    test_loss = 0
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        images = images.resize_(images.size()[0], 784)
        output = model.forward(images)
        test_loss += criterion(output, labels).item()
        ## Calculating the accuracy 
        # Model's output is log-softmax, take exponential to get the probabilities
        ps = torch.exp(output)
        equality = (labels.data == ps.max(1)[1])
        top_p, top_class = ps.topk(1, dim = 1)
        equality = top_class == labels.view(*top_class.shape)
        # Accuracy is number of correct predictions divided by all predictions, just take the mean
        accuracy += equality.float().mean()
    return test_loss, accuracy

In [10]:
# define function to build teacher models
def build_teacher_models(num_teachers, teacher_dropout):
    teacher_models = []
    for i in range(num_teachers):
        teacher_model = Classifier(784, 10, [512, 256, 128], drop_p = teacher_dropout)
        teacher_models.append(teacher_model)
    return teacher_models

In [11]:
# define function to train teacher models
def train_teacher_models(teacher_models, teacher_loaders, test_loader, teacher_criterion):
    for i in range(len(teacher_models)):
        print("Begin training Teacher", i+1)
        teacher_optimizer = optim.SGD(teacher_models[i].parameters(), lr=1e-4, momentum=0.9)
        train(teacher_models[i], teacher_loaders[i], test_loader, teacher_criterion, teacher_optimizer, epochs=50)
        print("Teacher",i+1,"trained successfully! \n")

In [ ]:
# define function to perform PATE analysis
def run_pate_analysis():
    data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
    print("Data Independent Epsilon:", data_ind_eps)
    print("Data Dependent Epsilon:", data_dep_eps)

In [14]:
## CREATE DATALOADERS
teacher_loaders = split_train_data(train_data, num_teachers=10)
test_loader, private_loader = split_test_data(test_data, ratio=0.8)

## DEFINE HYPERPARAMETERS
num_teachers = 10
teacher_dropout = 0.25 #[0.25, 0.22, 0.23, 0.34, 0.23, 0.21, 0.5, 0.1, 0.3, 0.12]
teacher_criterion = nn.NLLLoss()


## START MODELING
teacher_models = build_teacher_models(num_teachers, teacher_dropout)
train_teacher_models(teacher_models, teacher_loaders, test_loader, teacher_criterion)

Begin training Teacher 1


RuntimeError: CUDA error: unknown error

In [15]:
torch.cuda.is_available()

True